In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.utils import np_utils
from keras.layers import BatchNormalization
from keras.layers import Activation, Dropout
from multiprocessing import Process

import os
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
train_images = train_images.reshape(train_images.shape[0], train_images.shape[1] * train_images.shape[2]).astype('float32') / 255
test_images = test_images.reshape(test_images.shape[0], test_images.shape[1] * test_images.shape[2]).astype('float32') / 255

In [0]:
train_labels = np_utils.to_categorical(train_labels, 10)
test_labels = np_utils.to_categorical(test_labels, 10)

In [5]:
model1 = Sequential()
model1.add(Dense(128, input_dim=784, activation='relu'))
model1.add(Dense(10, activation='softmax'))

W0726 05:52:24.708668 139968611243904 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0726 05:52:24.745584 139968611243904 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 05:52:24.752850 139968611243904 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [6]:
model2 = Sequential()
model2.add(Dense(128, input_dim=784))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dense(10))
model2.add(BatchNormalization())
model2.add(Activation('softmax'))

W0726 05:52:24.881114 139968611243904 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.



In [7]:
model3 = Sequential()
model3.add(Dense(128, input_dim=784))
model3.add(Activation('relu'))
model3.add(Dropout(0.2))
model3.add(Dense(10))
model3.add(Activation('softmax'))

W0726 05:52:25.013097 139968611243904 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
models = []
models.append(model1)
models.append(model2)
models.append(model3)

In [9]:
for n_model in models:
  n_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

W0726 05:52:25.052902 139968611243904 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0726 05:52:25.080041 139968611243904 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [11]:
for n_model in models:
  n_model.fit(train_images, train_labels, epochs = 50, batch_size = 512)

W0726 05:52:25.387039 139968611243904 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
60000/60000 [==============================] - 4s 69us/step - loss: 0.7147 - acc: 0.7584
Epoch 2/50
60000/60000 [==============================] - 1s 8us/step - loss: 0.4628 - acc: 0.8416
Epoch 3/50
60000/60000 [==============================] - 0s 8us/step - loss: 0.4152 - acc: 0.8574
Epoch 4/50
60000/60000 [==============================] - 1s 8us/step - loss: 0.3871 - acc: 0.8657
Epoch 5/50
60000/60000 [==============================] - 0s 8us/step - loss: 0.3716 - acc: 0.8699
Epoch 6/50
60000/60000 [==============================] - 1s 9us/step - loss: 0.3511 - acc: 0.8761
Epoch 7/50
60000/60000 [==============================] - 1s 8us/step - loss: 0.3379 - acc: 0.8796
Epoch 8/50
60000/60000 [==============================] - 1s 8us/step - loss: 0.3257 - acc: 0.8836
Epoch 9/50
60000/60000 [==============================] - 0s 8us/step - loss: 0.3166 - acc: 0.8868
Epoch 10/50
60000/60000 [==============================] - 1s 8us/step - loss: 0.3081 - acc: 0.8889
Epoch 11

In [0]:
def TrainModel(model, X, Y, _epochs, _batch_size):
  model.fit(X, Y, epochs=_epochs, batch_size=_batch_size)

In [12]:
# 케라스는 스레드 세이프가 아니라고 하여서, 한개씩 fit 하였습니다.
# 멀티프로세싱으로 돌렸을 때는 0.11 정도의 accuracy가 나왔습니다.  아마 tensorflow로만 구현하면 더 빠른 fit을 위해 멀티프로세싱을 활용할 수 있을 것 같습니다.
'''
processes = []
for n_model in models:   # 모델들의 학습을 각각의 쓰레드에서 수행합니다.
  p = Process(target = TrainModel, args = (n_model, train_images, train_labels, 50, 512))
  p.start()
  processes.append(p)
for p in processes:        # join을 통하여 모든 프로세스가 작업을 끝낼때까지 대기합니다.
  p.join()
'''

'\nprocesses = []\nfor n_model in models:   # 모델들의 학습을 각각의 쓰레드에서 수행합니다.\n  p = Process(target = TrainModel, args = (n_model, train_images, train_labels, 50, 512))\n  p.start()\n  processes.append(p)\nfor p in processes:        # join을 통하여 모든 프로세스가 작업을 끝낼때까지 대기합니다.\n  p.join()\n'

In [13]:
for n_model in models:
  print("Accuracy : %.4f" % n_model.evaluate(test_images, test_labels)[1])

10000/10000 [==============================] - 0s 45us/step
Accuracy : 0.8925
10000/10000 [==============================] - 1s 60us/step
Accuracy : 0.8718
10000/10000 [==============================] - 0s 47us/step
Accuracy : 0.8922


In [34]:
labels = []
for n_model in models:
    predicts = np.argmax(n_model.predict(test_images), axis=1)
    labels.append(predicts)   # 각 model에서의 predicted label을 label에 넣습니다.
print(labels)

[array([9, 2, 1, ..., 8, 1, 5]), array([9, 2, 1, ..., 8, 1, 5]), array([9, 2, 1, ..., 8, 1, 5])]


In [35]:
import scipy
from scipy import stats

labels = np.array(labels)   
print(labels)
labels = np.transpose(labels, (1, 0))  # 각 column은 모델들이 특정 이미지에 대해 voting한 값들의 모임이 됩니다.
print(labels)
labels = scipy.stats.mode(labels, axis=1)[0]   # scipy.stats.mode => 세 모델이 내놓은 prediction 중 가장 보편적인 값을 찾습니다.
print(labels)
labels = np.squeeze(labels)   # 쓸모없는 차원을 제거합니다.
print(labels)

[[9 2 1 ... 8 1 5]
 [9 2 1 ... 8 1 5]
 [9 2 1 ... 8 1 5]]
[[9 9 9]
 [2 2 2]
 [1 1 1]
 ...
 [8 8 8]
 [1 1 1]
 [5 5 5]]
[[9]
 [2]
 [1]
 ...
 [8]
 [1]
 [5]]
[9 2 1 ... 8 1 5]


In [36]:
print(len(labels))
print(len(test_labels))
print(test_labels)
test_label = np.argmax(test_labels, axis=1)
print(test_label)

10000
10000
[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[9 2 1 ... 8 1 5]


In [37]:
cnt = 0
for i in range(len(labels)):
  if labels[i] == test_label[i]:
    cnt += 1
acc = cnt / len(labels)
print("Accuracy : %.4f" % acc)
# epoch가 작아서 그런지 미미하긴하지만, 세 모델들의 각 accuracy보다 큰 값이 나왔습니다.

Accuracy : 0.8973
